In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('All_Data.csv', sep=';')

# list_of_feats_Causal_Discovery = ['MD 210',
#  'DB 70DBD 4 ΘΕΡΜ ΚΟΛΕΚΤΕΡ ΨΥΓΕΙΩΝ (ΝΕΡΟ ΑΠΟ ΔΕΞΑΜΕΝΗ)',
#  'DB 30DBW 20 ramposition',
#  'DB 20DBW 174 EXTRACTION STEP',
#  'DB 10DBW 14 BACKWARD PRESS',
#  'DB 20DBD 292',
#  'Setpoint position exhaust damper',
#  'DB 400DBD 34 z2 energy',
#  'T 174',
#  'MD 284 C ana kw',
#  'MD 70 sinolo ypog-mpanioy', 'Timestamp', 'Energy']

# df = df[list_of_feats_Causal_Discovery]
df

,Timestamp,DB 10DBW 14 BACKWARD PRESS,DB 1260DBD 100 TOOL SLIDE POSITION,DB 13DBW 2,DB 30DBW 140 DISCHARGER POSITION,DB 30DBW 20 ramposition,DB 30DBW 80 v cont position,DB 371DBW 44 setpoint pump,MW 970,DB 10DBW 232,...,DB 75DBD 28,MD 210,MD 214,MD 70 sinolo ypog-mpanioy,T 0,C 22_Junker2,C 23_Junker2,C 24_Junker2,C 25_Junker2,Energy
0,2025-02-28 13:00:00,5.800000,635.218667,381.000000,2.2,1223.400000,20.133333,891.733333,15.800000,360.000000,...,7.266667,350.000000,450.000000,2736.063333,0.0,4.0,1.0,5.000000,2.0,101.671
1,2025-02-28 13:15:00,6.266667,508.268667,196.666667,52.8,1215.466667,15.533333,731.266667,15.866667,168.733333,...,1.600000,496.666667,716.666667,2731.106667,0.0,4.0,1.0,5.000000,2.0,153.854
2,2025-02-28 13:30:00,9.600000,381.356667,241.800000,52.8,803.866667,11.866667,826.933333,14.933333,108.666667,...,2.200000,146.666667,533.333333,2726.779333,0.0,4.0,1.0,5.000000,2.0,134.813
3,2025-02-28 13:45:00,9.133333,381.372000,168.466667,1.0,1031.133333,11.866667,830.400000,15.333333,123.133333,...,6.000000,0.000000,266.666667,2726.767333,0.0,4.0,1.0,5.000000,2.0,145.956
4,2025-02-28 14:00:00,6.600000,667.294667,-105.333333,104.6,954.800000,25.000000,943.466667,15.066667,169.000000,...,9.666667,293.333333,666.666667,2725.181333,0.0,4.0,1.0,4.466667,2.0,140.812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2286,2025-03-24 08:30:00,0.000000,0.490000,0.000000,1.0,82.666667,-3.000000,605.200000,10.000000,32.000000,...,0.000000,0.000000,0.000000,2776.373333,0.0,1.0,1.0,3.000000,1.0,31.785
2287,2025-03-24 08:45:00,0.000000,0.490000,0.000000,1.0,1226.000000,-2.000000,995.000000,10.000000,32.000000,...,0.000000,0.000000,0.000000,2776.691333,0.0,1.0,1.0,3.066667,1.0,16.260
2288,2025-03-24 09:00:00,0.000000,0.484667,0.000000,1.0,1226.000000,-2.000000,995.000000,10.000000,32.000000,...,0.000000,0.000000,0.000000,2776.366667,0.0,1.0,1.0,3.000000,1.0,17.297
2289,2025-03-24 09:15:00,0.000000,0.436667,-50.000000,1.0,1226.000000,-2.000000,995.000000,10.000000,32.000000,...,0.000000,0.000000,0.000000,2762.136667,0.0,1.0,1.0,3.000000,1.0,32.758


In [2]:
 # Categorical features
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index('Timestamp', inplace=True)
df['dow'] = df.index.day_name().str[:3]     # e.g. "Mon","Tue",…,"Sun"
df['hour'] = df.index.hour # 0–23
sun_low = df['dow'] == 'Sun' # basically all day on sunday
mon_low = (df['dow'] == 'Mon') & df['hour'].isin([0,1,2,3,4,5,6,7,8,9]) # basically from 0 to 9 on monday
wed_low = (df['dow'] == 'Wed') & df['hour'].isin([6,7,8,9,10,11,12,13,14]) # basically from 6 to 14 for maintenance on wednesday
sat_low = (df['dow'] == 'Sat') & df['hour'].isin([19,20,21,22,23]) # basically from 19 to 23 on saturday
df['is_low_usage'] = sun_low | mon_low | wed_low | sat_low
df.drop(columns=['dow', 'hour'], inplace=True)
df.reset_index(inplace=True)
df['is_low_usage'] = df['is_low_usage'].astype(int)
df['is_low_usage_next'] = df['is_low_usage'].shift(-1)




# cyclical encode timestamp as extra features
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:%M:%S')
df.set_index('Timestamp', inplace=True)
df['dow'] = df.index.day_name().str[:3]     # e.g. "Mon","Tue",…,"Sun"
df['hour'] = df.index.hour # 0–23
sun_low = df['dow'] == 'Sun' # basically all day on sunday
mon_low = (df['dow'] == 'Mon') & df['hour'].isin([0,1,2,3,4,5,6,7,8,9]) # basically from 0 to 9 on monday
wed_low = (df['dow'] == 'Wed') & df['hour'].isin([6,7,8,9,10,11,12,13,14]) # basically from 6 to 14 for maintenance on wednesday
sat_low = (df['dow'] == 'Sat') & df['hour'].isin([19,20,21,22,23]) # basically from 19 to 23 on saturday
df['is_low_usage'] = sun_low | mon_low | wed_low | sat_low
df.drop(columns=['dow', 'hour'], inplace=True)
df.reset_index(inplace=True)
df['is_low_usage'] = df['is_low_usage'].astype(int)



# Hour of day (0–23)
df['hour_sin'] = np.sin(2 * np.pi * df['Timestamp'].dt.hour / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['Timestamp'].dt.hour / 24)

# # Minute of hour (0–59)
# df['minute_sin'] = np.sin(2 * np.pi * df['Timestamp'].dt.minute / 60)
# df['minute_cos'] = np.cos(2 * np.pi * df['Timestamp'].dt.minute / 60)

# Compute number of days in each month
df['days_in_month'] = df['Timestamp'].dt.days_in_month
df['day_of_week_sin'] = np.sin(2 * np.pi * df['Timestamp'].dt.dayofweek / 7)
df['day_of_week_cos'] = np.cos(2 * np.pi * df['Timestamp'].dt.dayofweek / 7)
# Safe cyclic encoding
# df['day_sin'] = np.sin(2 * np.pi * df['Timestamp'].dt.day / df['days_in_month'])
# df['day_cos'] = np.cos(2 * np.pi * df['Timestamp'].dt.day / df['days_in_month'])
# Categorical features

# df['month_sin'] = np.sin(2 * np.pi * df['Timestamp'].dt.month / 12)
# df['month_cos'] = np.cos(2 * np.pi * df['Timestamp'].dt.month / 12)
df.drop(columns=['Timestamp', 'days_in_month'], inplace=True)
df.to_csv('energy_data.csv', index=False, sep=',')
df

,DB 10DBW 14 BACKWARD PRESS,DB 1260DBD 100 TOOL SLIDE POSITION,DB 13DBW 2,DB 30DBW 140 DISCHARGER POSITION,DB 30DBW 20 ramposition,DB 30DBW 80 v cont position,DB 371DBW 44 setpoint pump,MW 970,DB 10DBW 232,DB 10DBW 8 billet length,...,C 23_Junker2,C 24_Junker2,C 25_Junker2,Energy,is_low_usage,is_low_usage_next,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos
0,5.800000,635.218667,381.000000,2.2,1223.400000,20.133333,891.733333,15.800000,360.000000,670.0,...,1.0,5.000000,2.0,101.671,0,0.0,-0.258819,-0.965926,-0.433884,-0.900969
1,6.266667,508.268667,196.666667,52.8,1215.466667,15.533333,731.266667,15.866667,168.733333,670.0,...,1.0,5.000000,2.0,153.854,0,0.0,-0.258819,-0.965926,-0.433884,-0.900969
2,9.600000,381.356667,241.800000,52.8,803.866667,11.866667,826.933333,14.933333,108.666667,670.0,...,1.0,5.000000,2.0,134.813,0,0.0,-0.258819,-0.965926,-0.433884,-0.900969
3,9.133333,381.372000,168.466667,1.0,1031.133333,11.866667,830.400000,15.333333,123.133333,670.0,...,1.0,5.000000,2.0,145.956,0,0.0,-0.258819,-0.965926,-0.433884,-0.900969
4,6.600000,667.294667,-105.333333,104.6,954.800000,25.000000,943.466667,15.066667,169.000000,670.0,...,1.0,4.466667,2.0,140.812,0,0.0,-0.500000,-0.866025,-0.433884,-0.900969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2286,0.000000,0.490000,0.000000,1.0,82.666667,-3.000000,605.200000,10.000000,32.000000,810.0,...,1.0,3.000000,1.0,31.785,1,1.0,0.866025,-0.500000,0.000000,1.000000
2287,0.000000,0.490000,0.000000,1.0,1226.000000,-2.000000,995.000000,10.000000,32.000000,810.0,...,1.0,3.066667,1.0,16.260,1,1.0,0.866025,-0.500000,0.000000,1.000000
2288,0.000000,0.484667,0.000000,1.0,1226.000000,-2.000000,995.000000,10.000000,32.000000,810.0,...,1.0,3.000000,1.0,17.297,1,1.0,0.707107,-0.707107,0.000000,1.000000
2289,0.000000,0.436667,-50.000000,1.0,1226.000000,-2.000000,995.000000,10.000000,32.000000,810.0,...,1.0,3.000000,1.0,32.758,1,1.0,0.707107,-0.707107,0.000000,1.000000


In [3]:
import pandas as pd

df = pd.read_csv('energy_data.csv')
column_to_move = 'Energy'
df = df[[col for col in df.columns if col != column_to_move] + [column_to_move]]
df.to_csv('energy_data.csv', index=False)

In [4]:
len(df.columns)

57

In [5]:
import pandas as pd

df = pd.read_csv('energy_data.csv')
n = len(df)
train_end = int(n * 0.85)
X_train = df.iloc[:train_end]
X_train.to_csv('energy_data.csv', index=False)

In [6]:
len(X_train.columns)

57

In [7]:
X_train

,DB 10DBW 14 BACKWARD PRESS,DB 1260DBD 100 TOOL SLIDE POSITION,DB 13DBW 2,DB 30DBW 140 DISCHARGER POSITION,DB 30DBW 20 ramposition,DB 30DBW 80 v cont position,DB 371DBW 44 setpoint pump,MW 970,DB 10DBW 232,DB 10DBW 8 billet length,...,C 23_Junker2,C 24_Junker2,C 25_Junker2,is_low_usage,is_low_usage_next,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,Energy
0,5.800000,635.218667,381.000000,2.200000,1223.400000,20.133333,891.733333,15.800000,360.000000,670.0,...,1.0,5.000000,2.0,0,0.0,-0.258819,-9.659258e-01,-0.433884,-0.900969,101.671
1,6.266667,508.268667,196.666667,52.800000,1215.466667,15.533333,731.266667,15.866667,168.733333,670.0,...,1.0,5.000000,2.0,0,0.0,-0.258819,-9.659258e-01,-0.433884,-0.900969,153.854
2,9.600000,381.356667,241.800000,52.800000,803.866667,11.866667,826.933333,14.933333,108.666667,670.0,...,1.0,5.000000,2.0,0,0.0,-0.258819,-9.659258e-01,-0.433884,-0.900969,134.813
3,9.133333,381.372000,168.466667,1.000000,1031.133333,11.866667,830.400000,15.333333,123.133333,670.0,...,1.0,5.000000,2.0,0,0.0,-0.258819,-9.659258e-01,-0.433884,-0.900969,145.956
4,6.600000,667.294667,-105.333333,104.600000,954.800000,25.000000,943.466667,15.066667,169.000000,670.0,...,1.0,4.466667,2.0,0,0.0,-0.500000,-8.660254e-01,-0.433884,-0.900969,140.812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1942,6.733333,624.314667,-703.200000,104.600000,1202.400000,19.733333,865.266667,11.733333,264.066667,810.0,...,2.0,3.000000,3.0,0,0.0,-1.000000,-1.836970e-16,0.433884,-0.900969,172.808
1943,0.933333,254.330667,559.066667,25.866667,1105.733333,7.333333,992.466667,10.800000,32.000000,810.0,...,2.0,3.000000,3.0,0,0.0,-1.000000,-1.836970e-16,0.433884,-0.900969,149.665
1944,0.000000,0.490000,0.000000,1.000000,1220.000000,-1.000000,988.666667,11.000000,216.066667,810.0,...,2.0,3.000000,3.0,0,0.0,-0.965926,2.588190e-01,0.433884,-0.900969,55.749
1945,1.666667,396.317333,217.133333,104.600000,929.066667,14.533333,854.466667,11.466667,264.066667,810.0,...,2.0,3.000000,3.0,0,0.0,-0.965926,2.588190e-01,0.433884,-0.900969,122.805
